# Setup



In [1]:
!pip install -q mistral_inference


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 108.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.1 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
import time
from tqdm import tqdm

import pandas as pd
from huggingface_hub import snapshot_download
from pathlib import Path

from mistral_inference.transformer import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/xformers/__init__.py", line 57, in _is_triton_available
    import triton  # noqa
ModuleNotFoundError: No module named 'triton'


# Dataset

In [4]:
!wget https://raw.githubusercontent.com/mistralai/cookbook/refs/heads/main/data/Symptom2Disease.csv

--2025-01-03 18:44:21--  https://raw.githubusercontent.com/mistralai/cookbook/refs/heads/main/data/Symptom2Disease.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 229852 (224K) [text/plain]
Saving to: ‘Symptom2Disease.csv’

Symptom2Disease.csv 100%[===================>] 224.46K  --.-KB/s    in 0.004s  

2025-01-03 18:44:22 (49.8 MB/s) - ‘Symptom2Disease.csv’ saved [229852/229852]



In [5]:
data = pd.read_csv("Symptom2Disease.csv", index_col=False)
df_shuffled = data.sample(frac=1, random_state=42)
df_shuffled.iloc[0]["text"]

"I have respiratory issues, especially when doing out. Unexpectedly, I'm sweating and flushed. I frequently have yeast infections and urinary tract infections"

# Mistral-7B-Instruct-v0.3

In [6]:
SYSTEM_PROMPT = """
You are an expert medical professional named Medical AI.

Using the text provided by a user which is explaining symptoms of a disease, give your diagnosis.

You have to choose from one of the diseases below:

Psoriasis
Varicose Veins
peptic ulcer disease
drug reaction
gastroesophageal reflux disease
allergy
urinary tract infection
Malaria
Jaundice
Cervical spondylosis
Migraine
Hypertension
Bronchial Asthma
Acne
Arthritis
Dimorphic Hemorrhoids
Pneumonia
Common Cold
Fungal infection
Dengue
Impetigo
Chicken pox
Typhoid
diabetes

Do not add any other text except the disease name.

For example:

User input: The skin around my mouth, nose, and eyes is red and inflamed. It is often itchy and uncomfortable. There is a noticeable inflammation in my nails
Medical AI: Psoriasis

User input:{{prompt}}
Medical AI:
"""

In [7]:
mistral_models_path = Path.home().joinpath('mistral_models', '7B-Instruct-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

consolidated.safetensors:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

tokenizer.model.v3:   0%|          | 0.00/587k [00:00<?, ?B/s]

params.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

'/root/mistral_models/7B-Instruct-v0.3'

In [9]:
tokenizer = MistralTokenizer.from_file(f"{mistral_models_path}/tokenizer.model.v3")
model = Transformer.from_folder(mistral_models_path, device = "cuda")

In [10]:
# Create a list to store results
results = []

# Iterate through prompts with progress bar
for index, row in tqdm(df_shuffled.iterrows(), total=len(df_shuffled)):
    try:
        # Get LLM response - modify this part according to your LLM API
        # I am using Mistral 7B-Instruct-v0.3
        prompt = SYSTEM_PROMPT.replace("{{prompt}}", row["text"])
        completion_request = ChatCompletionRequest(messages=[UserMessage(content=prompt)])
        tokens = tokenizer.encode_chat_completion(completion_request).tokens

        out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
        response_text = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])


        # Store original prompt and response
        results.append({
            'prompt': row['text'],
            'response': response_text
        })

        print(response_text)

        # Optional: Add delay to respect rate limits
        time.sleep(0.1)

    except Exception as e:
        print(f"Error processing prompt at index {index}: {str(e)}")
        # Store error in results
        results.append({
            'prompt': row['prompt'],
            'response': f"ERROR: {str(e)}"
        })

# Create new dataframe with results
results_df = pd.DataFrame(results)

# Save to CSV
results_df.to_csv('llm_responses.csv', index=False)

  0%|          | 1/1200 [00:02<58:21,  2.92s/it]

Bronchial Asthma, Fungal Infection, Urinary Tract Infection


  0%|          | 2/1200 [00:03<29:27,  1.48s/it]

Malaria


  0%|          | 3/1200 [00:04<25:55,  1.30s/it]

...

Gastroesophageal Reflux Disease (GERD)


100%|█████████▉| 1199/1200 [11:39<00:00,  1.61it/s]

Dengue


100%|██████████| 1200/1200 [11:39<00:00,  1.71it/s]

Peptic Ulcer Disease
